In [1]:
import requests
import pandas as pd

from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
mars_news = 'https://mars.nasa.gov/news/'
mars_pics = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
mars_facts = 'https://space-facts.com/mars/'
mars_hemisp = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [3]:
executable_path = {'executable_path':ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [C:\Users\theko\.wdm\drivers\chromedriver\win32\84.0.4147.30\chromedriver.exe] found in cache
 


# Scrape News

In [4]:
browser.visit(mars_news)
browser.driver.maximize_window()
soup_mars_news = BeautifulSoup(browser.html, 'lxml')

In [5]:
list_soup_mars_news = soup_mars_news.find_all('li', class_='slide')
mars_news_list = []
for l in list_soup_mars_news:
    try:
        mars_news_list.append({'title':l.find('div', class_='content_title').text.strip(),\
                                'content':l.find('div', class_='article_teaser_body').text.strip(),\
                                'date':l.find('div', class_='list_date').text.strip()})
    except:
        pass

In [6]:
len(mars_news_list)

40

# Scrape Pictures

In [7]:
browser.visit(mars_pics)
browser.driver.maximize_window()
soup_mars_pics = BeautifulSoup(browser.html, 'lxml')

In [8]:
list_soup_mars_pics = soup_mars_pics.find_all('ul', class_='articles')[0].find_all('li', class_='slide')
mars_pics_web = 'https://www.jpl.nasa.gov'
mars_pics_list=[]
for l in list_soup_mars_pics:
    try:
        release_date = {'release_date':l.find(class_='release_date').text}
        item_tease_overlay = {'item_tease_overlay':l.find(class_='item_tease_overlay').text}
        thumb = {'thumb':mars_pics_web+l.find(class_='thumb')['src']}
        large = {'large':mars_pics_web+l.a['data-fancybox-href']}
        mars_pics_list.append(dict(release_date, **item_tease_overlay, **thumb, **large))
    except:
        pass

In [9]:
len(mars_pics_list)

32

# Scrape Facts Tables

In [10]:

list_mars_facts = pd.read_html(mars_facts)

mars_facts_list = []
for l in list_mars_facts:
    mars_facts_list.append({'fact_html_table' : l.to_html(index=False)})


In [11]:
list_mars_facts[1]

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days
5,Temperature:,-87 to -5 °C,-88 to 58°C


# Scrape Hemisphere Pictures

In [12]:
browser.visit(mars_hemisp)
browser.driver.maximize_window()
soup_mars_hemisp = BeautifulSoup(browser.html, 'lxml')

In [13]:
list_soup_mars_hempisp = soup_mars_hemisp.find_all('div', class_='collapsible results')[0].find_all('div', class_='item')
mars_hemisp_web = 'https://astrogeology.usgs.gov'
mars_hemisp_list = []
for l in list_soup_mars_hempisp:
    try:
        title = l.h3.text.replace(' Enhanced', '')
        link = BeautifulSoup(requests.get(mars_hemisp_web+l.a['href']).text, 'lxml')
        img_url = link.find_all('div', class_='downloads')[0].a['href']
        mars_hemisp_list.append({'title':title, 'img_url':img_url})
    except:
        pass

In [14]:
len(mars_hemisp_list)

4

In [15]:
browser.driver.close()